In [1]:
import numpy as np 
import pandas as pd
import datetime
import calendar
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', None)
%matplotlib inline

In [2]:
X_train = pd.read_csv('../data/X_train.csv', sep=',', encoding='utf-8')
target = pd.read_csv('../data/target.csv', sep=',', encoding='utf-8')
X_test = pd.read_csv('../data/X_test.csv', sep=',', encoding='utf-8')

In [3]:
print('train shape:', X_train.shape,'| target shape:', target.shape)
print('test shape:', X_test.shape)

train shape: (5998506, 47) | target shape: (5998506, 1)
test shape: (100000, 48)


In [4]:
X_test.duplicated('id').any()

False

In [5]:
ids = X_test[['id']]
X_test.drop('id', axis=1, inplace=True)

In [6]:
print('train shape:', X_train.shape,'| target shape:', target.shape)
print('test shape:', X_test.shape)

train shape: (5998506, 47) | target shape: (5998506, 1)
test shape: (100000, 47)


In [7]:
ids.to_csv('test_.csv', sep=',', encoding='utf-8')

In [8]:
X_test.shape

(100000, 47)

In [9]:
print(len(target[target['sepostulo'] == False]))
print(len(target[target['sepostulo'] == True]))

2998772
2999734


In [10]:
X_train.head(1)

,Administración,AdministracióndePersonal,Almacén/Depósito/Expedición,AtenciónalCliente,CallCenter,Comercial,Contabilidad,CréditosyCobranzas,Gastronomia,Legal,Logística,MantenimientoyLimpieza,Marketing,Producción,Recepcionista,RecursosHumanos,Secretaria,Telemarketing,Tesorería,Ventas,descripcion,edad,estado_estudio_Abandonado,estado_estudio_EnCurso,estado_estudio_Graduado,idaviso,idpostulante,nivel_laboral_Gerencia/AltaGerencia/Dirección,nivel_laboral_Jefe/Supervisor/Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior/Semi-Senior,nombre_area,nombre_estudio_Doctorado,nombre_estudio_Master,nombre_estudio_Otro,nombre_estudio_Posgrado,nombre_estudio_Secundario,nombre_estudio_Terciario/Tecnico,nombre_estudio_Universitario,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,tipo_full,tipo_otro,tipo_part,titulo
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,"[('caba', 2), ('importante', 1), ('cadena', 1)...",27,0,0,1,1112418025,6r5vxAM,0,0,0,0,1,Ventas,0,0,0,0,0,0,1,1,0,0,True,False,False,"['j', 'cajeros', 'zona', 'caba']"


In [11]:
X_test.head(1)

,idaviso,idpostulante,edad,nombre_estudio_Doctorado,nombre_estudio_Master,nombre_estudio_Otro,nombre_estudio_Posgrado,nombre_estudio_Secundario,nombre_estudio_Terciario/Tecnico,nombre_estudio_Universitario,estado_estudio_Abandonado,estado_estudio_EnCurso,estado_estudio_Graduado,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,titulo,descripcion,nombre_area,tipo_full,tipo_part,tipo_otro,nivel_laboral_Gerencia/AltaGerencia/Dirección,nivel_laboral_Jefe/Supervisor/Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior/Semi-Senior,Ventas,Administración,Producción,Comercial,AtenciónalCliente,Recepcionista,CallCenter,Tesorería,Telemarketing,MantenimientoyLimpieza,Almacén/Depósito/Expedición,Contabilidad,RecursosHumanos,Gastronomia,Logística,Secretaria,AdministracióndePersonal,CréditosyCobranzas,Marketing,Legal
0,739260,6M9ZQR,42,1,0,0,0,0,0,0,0,0,1,1,0,0,"['asistente', 'comercial']","[('p', 2), ('cliente', 1), ('empresa', 1), ('n...",Comercial,True,False,False,0,1,0,0,0,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


## TEMPORAL: se eliminan las columnas con los arrays, decidir que hacer con este dato

In [12]:
X_train.drop(columns=['descripcion', 'titulo'], inplace=True)
X_test.drop(columns=['descripcion', 'titulo'], inplace=True)

In [13]:
X_train.head(1)

,Administración,AdministracióndePersonal,Almacén/Depósito/Expedición,AtenciónalCliente,CallCenter,Comercial,Contabilidad,CréditosyCobranzas,Gastronomia,Legal,Logística,MantenimientoyLimpieza,Marketing,Producción,Recepcionista,RecursosHumanos,Secretaria,Telemarketing,Tesorería,Ventas,edad,estado_estudio_Abandonado,estado_estudio_EnCurso,estado_estudio_Graduado,idaviso,idpostulante,nivel_laboral_Gerencia/AltaGerencia/Dirección,nivel_laboral_Jefe/Supervisor/Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior/Semi-Senior,nombre_area,nombre_estudio_Doctorado,nombre_estudio_Master,nombre_estudio_Otro,nombre_estudio_Posgrado,nombre_estudio_Secundario,nombre_estudio_Terciario/Tecnico,nombre_estudio_Universitario,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,tipo_full,tipo_otro,tipo_part
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,27,0,0,1,1112418025,6r5vxAM,0,0,0,0,1,Ventas,0,0,0,0,0,0,1,1,0,0,True,False,False


In [14]:
X_test.head(1)

,idaviso,idpostulante,edad,nombre_estudio_Doctorado,nombre_estudio_Master,nombre_estudio_Otro,nombre_estudio_Posgrado,nombre_estudio_Secundario,nombre_estudio_Terciario/Tecnico,nombre_estudio_Universitario,estado_estudio_Abandonado,estado_estudio_EnCurso,estado_estudio_Graduado,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,nombre_area,tipo_full,tipo_part,tipo_otro,nivel_laboral_Gerencia/AltaGerencia/Dirección,nivel_laboral_Jefe/Supervisor/Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior/Semi-Senior,Ventas,Administración,Producción,Comercial,AtenciónalCliente,Recepcionista,CallCenter,Tesorería,Telemarketing,MantenimientoyLimpieza,Almacén/Depósito/Expedición,Contabilidad,RecursosHumanos,Gastronomia,Logística,Secretaria,AdministracióndePersonal,CréditosyCobranzas,Marketing,Legal
0,739260,6M9ZQR,42,1,0,0,0,0,0,0,0,0,1,1,0,0,Comercial,True,False,False,0,1,0,0,0,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [24]:
# Solo para probar un submit elimino las columnas que no son numericas/booleanas
X_train.drop(columns=['nombre_area'], inplace=True)
X_test.drop(columns=['nombre_area'], inplace=True)

In [16]:
X_train.drop(columns=['idpostulante'], inplace=True)
X_test.drop(columns=['idpostulante'], inplace=True)

In [17]:
# Ordenamos las columnas para que queden igual
X_train = X_train.reindex_axis(sorted(X_train.columns), axis=1)
X_test = X_test.reindex_axis(sorted(X_test.columns), axis=1)

/home/tbotalla/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  
/home/tbotalla/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
print(X_train.isnull().values.any())
print(X_test.isnull().values.any())

True
False


In [19]:
X_train.head(1)

,Administración,AdministracióndePersonal,Almacén/Depósito/Expedición,AtenciónalCliente,CallCenter,Comercial,Contabilidad,CréditosyCobranzas,Gastronomia,Legal,Logística,MantenimientoyLimpieza,Marketing,Producción,Recepcionista,RecursosHumanos,Secretaria,Telemarketing,Tesorería,Ventas,edad,estado_estudio_Abandonado,estado_estudio_EnCurso,estado_estudio_Graduado,idaviso,nivel_laboral_Gerencia/AltaGerencia/Dirección,nivel_laboral_Jefe/Supervisor/Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior/Semi-Senior,nombre_area,nombre_estudio_Doctorado,nombre_estudio_Master,nombre_estudio_Otro,nombre_estudio_Posgrado,nombre_estudio_Secundario,nombre_estudio_Terciario/Tecnico,nombre_estudio_Universitario,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,tipo_full,tipo_otro,tipo_part
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,27,0,0,1,1112418025,0,0,0,0,1,Ventas,0,0,0,0,0,0,1,1,0,0,True,False,False


In [20]:
X_test.head(1)

,Administración,AdministracióndePersonal,Almacén/Depósito/Expedición,AtenciónalCliente,CallCenter,Comercial,Contabilidad,CréditosyCobranzas,Gastronomia,Legal,Logística,MantenimientoyLimpieza,Marketing,Producción,Recepcionista,RecursosHumanos,Secretaria,Telemarketing,Tesorería,Ventas,edad,estado_estudio_Abandonado,estado_estudio_EnCurso,estado_estudio_Graduado,idaviso,nivel_laboral_Gerencia/AltaGerencia/Dirección,nivel_laboral_Jefe/Supervisor/Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior/Semi-Senior,nombre_area,nombre_estudio_Doctorado,nombre_estudio_Master,nombre_estudio_Otro,nombre_estudio_Posgrado,nombre_estudio_Secundario,nombre_estudio_Terciario/Tecnico,nombre_estudio_Universitario,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,tipo_full,tipo_otro,tipo_part
0,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,42,0,0,1,739260,0,1,0,0,0,Comercial,1,0,0,0,0,0,0,1,0,0,True,False,False


In [21]:
target['sepostulo']

0           True
1           True
2          False
3           True
4           True
5          False
6           True
7          False
8           True
9          False
10         False
11         False
12          True
13          True
14         False
15          True
16          True
17         False
18          True
19          True
20          True
21         False
22         False
23         False
24          True
25         False
26         False
27          True
28          True
29          True
           ...  
5998476    False
5998477    False
5998478     True
5998479    False
5998480    False
5998481     True
5998482     True
5998483     True
5998484    False
5998485     True
5998486     True
5998487     True
5998488    False
5998489    False
5998490     True
5998491     True
5998492     True
5998493     True
5998494     True
5998495    False
5998496     True
5998497     True
5998498     True
5998499    False
5998500     True
5998501    False
5998502     True
5998503     Tr

# Cross-validation

In [21]:
from sklearn.model_selection import cross_val_score, KFold
# seed para generar los mismos fold en cada cv
seed = 7
kfold = KFold(3,random_state=seed) # 3 sino tarda demasiado
scoring = 'roc_auc'
def cv_score(model,X,y,cv):
    #return cross_val_score(model,X,y,n_jobs=-1,cv=cv,scoring=scoring,verbose=3).mean()
    return cross_val_score(model,X,y,n_jobs=1,cv=cv,scoring=scoring,verbose=3).mean()

**Standardization & normalization**

In [25]:
from sklearn.preprocessing import scale, normalize
X_train_std = scale(X_train)
X_test_std = scale(X_test)
X_train_nrm = normalize(X_train)
X_test_nrm = normalize(X_test)

/home/tbotalla/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


### Rankeo de algoritmos

In [ ]:
%%time
'''from sklearn.linear_model import LinearRegression, Ridge, BayesianRidge, SGDRegressor, Lasso, LassoLars, Lars, ElasticNet, PassiveAggressiveRegressor, RANSACRegressor, HuberRegressor, OrthogonalMatchingPursuit
from sklearn.svm import LinearSVR
from sklearn.cross_decomposition import PLSRegression
from sklearn.tree import DecisionTreeRegressor
#from sklearn.neighbors import KNeighborsRegressor
#from sklearn.kernel_ridge import KernelRidge
#from sklearn.gaussian_process import GaussianProcessRegressor
#from sklearn.isotonic import IsotonicRegression
'''
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

models = []
models.append(('DecTree',DecisionTreeClassifier()))
models.append(('LogReg',LogisticRegression()))
models.append(('KNN',KNeighborsClassifier()))
models.append(('LDA',LinearDiscriminantAnalysis()))
models.append(('GaussianNB',GaussianNB()))
#models.append(('SVC',SVC())) # no termina


scores = []
for name,model in models:
    print('################################',name,'################################')
    score = cv_score(model, X_train, target.values.ravel(), cv=kfold)
    '''
    if(name in ['LR','OMP','Lars','BayRidge',]):
        score = cv_score(model,X_train_nrm,y_target,cv=kfold)
    else:
        score = cv_score(model,X_train_std,y_target,cv=kfold)
    '''    
    scores.append((name,score))

################################ LogReg ################################
[CV]  ................................................................
[CV] ....................... , score=0.5102176214256832, total=  12.6s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.7s remaining:    0.0s


[CV] ....................... , score=0.5092362135973157, total=  11.4s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   24.1s remaining:    0.0s


[CV] ....................... , score=0.5097576417814326, total=  11.4s
################################ KNN ################################
[CV]  ................................................................


...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.0s remaining:    0.0s


[CV] ....................... , score=0.5436817289961993, total=  11.9s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   23.9s remaining:    0.0s


[CV] ....................... , score=0.5442461833938491, total=  11.9s
################################ SVC ################################
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   35.8s finished


**El que mejor score da es KNN**

In [22]:
def predict(model,X_train,X_test,y):
    model.fit(X_train,y.values.ravel())
    return model.predict(X_test)

def pred_to_csv(prediction,name):
    test = pd.read_csv('test_.csv')
    df = pd.DataFrame(prediction,columns={'sepostulo'})
    df['id'] = test.id
    df = df.loc[:,['id','sepostulo']]
    df.sepostulo = df.sepostulo.astype(int)
    df.to_csv('pred_' + name + '.csv',index=False)

In [57]:
from sklearn.linear_model import SGDClassifier

pred = predict(SGDClassifier(), X_train, X_test, target)
pred_to_csv(pred,'SGD')

/home/tbotalla/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [25]:
from sklearn.tree import DecisionTreeClassifier

pred = predict(DecisionTreeClassifier(), X_train, X_test, target)
pred_to_csv(pred,'DecisionTreeClassifier') # 0.7 score en kaggle

In [26]:
from sklearn.neighbors import KNeighborsClassifier

pred = predict(KNeighborsClassifier(), X_train, X_test, target)
pred_to_csv(pred,'KNeighborsClassifier') # 0.76 score en kaggle

In [8]:
from sklearn.model_selection import cross_val_score, KFold
# seed para generar los mismos fold en cada cv
seed = 7
kfold = KFold(10,random_state=seed)
scoring = 'neg_mean_squared_error'
#scoring = 'jaccard_similarity_score'
def cv_score(model,X,y,cv):
#    MSE_score = cross_val_score(model,X,y,n_jobs=-1,cv=cv,scoring='neg_mean_squared_error').mean()
    return cross_val_score(model,X,y,n_jobs=-1,cv=cv,scoring=scoring,verbose=3).mean()
#    print('params:\n',model.get_params())
#    print('MSE score:\n',MSE_score)

In [ ]:
from sklearn.preprocessing import scale, normalize
X_train_std = scale(X_train)
X_test_std = scale(X_test)
#X_train_nrm = normalize(X_train)
#X_test_nrm = normalize(X_test)

In [ ]:
%%time
models = []
models.append(('DecTree',tree.DecisionTreeClassifier()))
scores = []
for name,model in models:
    print('################################',name,'################################')
    score = cv_score(model,X_train,target,cv=kfold)
    scores.append((name,score))

################################ DecTree ################################
